<a href="https://colab.research.google.com/github/larissavaladao/time_series_curuai/blob/main/6.%20parameters_by_period/1.set_periods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>